Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'

REPO_ROOT = "/home/leon/models/NeuroFlame"
pal = sns.color_palette("tab10")
```

Imports
=======

``` ipython
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
from scipy.stats import binned_statistic
```

``` ipython
import sys
sys.path.insert(0, '../../../')

import pandas as pd
import torch.nn as nn
from time import perf_counter
from scipy.stats import circmean

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump, circcvl, decode_bump_torch
from src.lr_utils import masked_normalize, clamp_tensor, normalize_tensor
from src.utils import clear_cache
```

``` ipython
import pickle as pkl
import os

def pkl_save(obj, name, path="."):
    os.makedirs(path, exist_ok=True)
    destination = path + "/" + name + ".pkl"
    print("saving to", destination)
    pkl.dump(obj, open(destination, "wb"))


def pkl_load(name, path="."):
    source = path + "/" + name + '.pkl'
    print('loading from', source)
    return pkl.load(open( source, "rb"))

```

Helpers
=======

``` ipython
def map2center(angles):
    """Map angles from [0, 2π] to [-π, π] using PyTorch tensors."""
    return np.where(angles > np.pi, angles - 2 * np.pi, angles)

def map2pos(angles):
    """Map angles from [-π, π] to [0, 2π] using PyTorch tensors."""
    return np.where(angles < 0, angles + 2 * np.pi, angles)
```

``` ipython
def maptens2center(angles):
    """Map angles from [0, 2π] to [-π, π] using PyTorch tensors."""
    return torch.where(angles > torch.pi, angles - 2 * torch.pi, angles)

def maptens2pos(angles):
    """Map angles from [-π, π] to [0, 2π] using PyTorch tensors."""
    return torch.where(angles < 0, angles + 2 * torch.pi, angles)
```

``` ipython
def add_vlines(model, ax=None):

    if ax is None:
        for i in range(len(model.T_STIM_ON)):
            plt.axvspan(model.T_STIM_ON[i], model.T_STIM_OFF[i], alpha=0.25)
    else:
        for i in range(len(model.T_STIM_ON)):
            ax.axvspan(model.T_STIM_ON[i], model.T_STIM_OFF[i], alpha=0.25)

```

``` ipython
import torch
import numpy as np

def generate_weighted_phase_samples(N_BATCH, angles, preferred_angle, sigma):
    # Convert angles list to a tensor
    angles_tensor = torch.tensor(angles)

    # Calculate Gaussian probability distribution centered at preferred_angle
    probs = np.exp(-0.5 * ((angles - preferred_angle) / sigma) ** 2)
    probs /= probs.sum()  # Normalize to get probabilities

    # Create a categorical distribution from the computed probabilities
    distribution = torch.distributions.Categorical(torch.tensor(probs))

    # Sample from the distribution
    indices = distribution.sample((N_BATCH,))

    # Map indices to angles and reshape to (N_BATCH, 1)
    phase_samples = angles_tensor[indices].reshape(N_BATCH, 1)

    return phase_samples
```

``` ipython
import torch
import numpy as np
import matplotlib.pyplot as plt

def continuous_biased_phases(N_BATCH, preferred_angle, sigma):
    # Generate samples from a normal distribution using PyTorch
    phase_samples = torch.normal(mean=preferred_angle, std=sigma, size=(N_BATCH, 1))

    # Normalize angles to the range [0, 360)
    phase_samples = phase_samples % 360

    return phase_samples
```

``` ipython
import torch
import numpy as np
import matplotlib.pyplot as plt

def continuous_bimodal_phases(N_BATCH, preferred_angle, sigma):
    # Sample half from preferred_angle and half from preferred_angle + 180
    half_batch = N_BATCH // 2

    # Sample from preferred_angle
    samples_1 = torch.normal(mean=preferred_angle, std=sigma, size=(half_batch, 1))

    # Sample from preferred_angle + 180
    samples_2 = torch.normal(mean=(preferred_angle + 180) % 360, std=sigma, size=(N_BATCH - half_batch, 1))

    # Combine samples and wrap around 360
    phase_samples = torch.cat((samples_1, samples_2), dim=0) % 360

    return phase_samples
```

``` ipython
def continuous_bimodal_phases(N_BATCH, reference, sigma):

    ref_copy = reference.clone()
    idx = torch.rand(N_BATCH, 1) > 0.5
    ref_copy[idx] = (reference[idx] + 180.0) % 360.0
    samples = ref_copy + sigma * torch.randn(N_BATCH, 1)

    return samples % 360.0
```

Model
=====

``` ipython
kwargs = {
    'GAIN': 1.0,
    'DURATION': 6.0,
    'T_STEADY': 4,

    'T_STIM_ON': [1.0, 5.0],
    'T_STIM_OFF': [2.0, 6.0],

    'Jab': [1.0, -1.3, 1, -1],
    'I0': [0.5, -2.0],
    'PHI0': [180.0, 180],
    'SIGMA0': [2.0, 0.0],
    'M0': 1,

    'RANDOM_DELAY': 0,
    'MIN_DELAY': 0,
    'MAX_DELAY': 3,

    'IF_FF_STP': 0,
    'FF_USE': 0.5,
    'TAU_FF_FAC': 0.0,
    'TAU_FF_REC': 0.5,

    'IS_STP': [1, 0, 0, 0],
    'USE': [0.05, 0.03, 0.03, 0.1],
    'TAU_FAC': [2.0, 2.0, 2.0, 0.0],
    'TAU_REC': [0.2, 0.2, 0.2, 0.1],
    'W_STP': [1.0, 3.0, 4.0, 1.0],

    'IF_NMDA': 0,
    'R_NMDA': 1.0,
    'TAU_NMDA': [2.0, 2.0],

    'IF_FF_ADAPT': 0,
    'A_FF_ADAPT': 1.0,
    'TAU_FF_ADAPT': 100.0,

    'IF_ADAPT': 1,
    'A_ADAPT': 1.0,
    'TAU_ADAPT': 150.0,
}
```

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
conf_name = "train_odr_EI.yml"
DEVICE = 'cuda:1'
# seed = np.random.randint(0, 1e6)

seed = 0
print('seed', seed)
```

``` ipython
N_BATCH = 128 * 8
print(N_BATCH)
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE=DEVICE, SEED=seed, N_BATCH=N_BATCH, **kwargs)
```

``` ipython
model_state_dict = torch.load('../models/odr/odr_%d.pth' % seed)
model.load_state_dict(model_state_dict);
model.eval();
```

``` ipython
#model.J_STP =torch.nn.Parameter(1.15 * model.J_STP.detach())
```

Simulating Consecutive Trials
=============================

``` ipython
reference = 360.0 * torch.rand(N_BATCH, 1)
sigma = 45
```

``` ipython
model.N_BATCH = N_BATCH

model.PHI0 = torch.zeros(size=(N_BATCH, len(model.I0), 1), device=DEVICE, dtype=torch.float)
model.PHI0[:, 0] = continuous_bimodal_phases(N_BATCH, reference, sigma)

print(reference.shape, model.PHI0.shape)

with torch.no_grad():
    ff_input = model.init_ff_input()
    rates_tensor = model.forward(ff_input=ff_input)
    del ff_input
    clear_cache()

print(rates_tensor.shape)
```

``` ipython
num_epochs = 100

rates_list = []
prev_list = [model.PHI0[:, 0].cpu().detach()]
curr_list = []

for epoch in tqdm(range(num_epochs)):
    with torch.no_grad():

        model.PHI0 = torch.zeros(size=(N_BATCH, len(model.I0), 1), device=DEVICE, dtype=torch.float)
        model.PHI0[:, 0] = continuous_bimodal_phases(N_BATCH, reference, sigma)

        ff_input = model.init_ff_input()
        rates = model.forward(ff_input=ff_input, IF_INIT=0)

        curr_list.append(model.PHI0[:, 0].cpu().detach())
        prev_list.append(model.PHI0[:, 0].cpu().detach())

        rates_list.append(rates.cpu().detach())

        del ff_input, model.PHI0, rates
        clear_cache()

rates_list = torch.stack(rates_list).cpu().numpy()
prev_list = torch.stack(prev_list).cpu().numpy()[:-1]
curr_list = torch.stack(curr_list).cpu().numpy()

print('rates', rates_list.shape)
print('curr', curr_list.shape, 'prev', prev_list.shape)
```

``` ipython
DURATION = rates_list.shape[2] / 10
N_NEURONS = rates_list.shape[-1]
N_TRIALS = rates_list.shape[0]
N_SESSION = rates_list.shape[1]
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].hist(curr_list[:, 0, 0] * 180 / np.pi, bins=20, histtype='step')
ax[0].set_xlabel('Stimulus Loc. (°)')
ax[0].set_ylabel('Count')

ax[1].hist(reference[:, 0] * 180.0 / np.pi, bins=20, histtype='step')
ax[1].set_xlabel('Reference Loc. (°)')
ax[1].set_ylabel('Count')

plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 1, figsize=[2.5*width, height])

n_trials = 10
idx = np.random.randint(0, N_SESSION)
rates = np.vstack(rates_list[:n_trials, idx]).T

plt.imshow(rates, aspect='auto', cmap='jet', vmin=0, vmax=5, origin='lower', extent=[0, n_trials * DURATION, 0, N_NEURONS])
plt.ylabel('Pref. Location (°)')
plt.yticks(np.linspace(0, N_NEURONS, 5), np.linspace(0, 360, 5).astype(int))
plt.xlabel('Time (s)')

plt.show()
```

``` ipython
```

Errors
======

``` ipython
n_half = num_epochs // 2
#n_half = 1
```

``` ipython
curr_ini =  curr_list[:n_half]
curr_last = curr_list[-n_half:]

prev_ini =  prev_list[:n_half]
prev_last = prev_list[-n_half:]
print(curr_ini.shape, prev_ini.shape)
```

``` ipython
_, _, phi_ini = decode_bump_torch(rates_list[:n_half, ...], axis=-1)
print(phi_ini.shape)
```

``` ipython
_, _, phi_last = decode_bump_torch(rates_list[-n_half:, ...], axis=-1)
print(phi_last.shape)
```

``` ipython
def get_error_curr_prev(phi, curr, prev, reference):
    target_loc = curr  * 180.0 / np.pi

    rel_loc = prev - curr
    rel_loc = (rel_loc + np.pi) % (2 * np.pi) - np.pi
    rel_loc *= 180 / np.pi

    ref_loc = reference[np.newaxis] - curr
    ref_loc = (ref_loc + np.pi) % (2 * np.pi) - np.pi
    ref_loc *= 180 / np.pi

    error_curr = phi - curr
    error_curr = (error_curr + np.pi) % (2 * np.pi) - np.pi
    error_curr *= 180 / np.pi

    return np.vstack(target_loc), np.vstack(rel_loc), np.vstack(ref_loc), np.array(error_curr)
```

``` ipython
def get_end_point(model, errors):

    stim_start_idx = ((model.start_indices - model.N_STEADY) / model.N_WINDOW).to(int).cpu().numpy()

    end_point = []
    for k in range(errors.shape[1]):
            idx = stim_start_idx[1][k]-1
            end_point.append(errors[:, k, idx])

    return np.array(end_point).T.reshape(-1, 1)
```

``` ipython
targ_ini, rel_ini, ref_ini, errors_ini = get_error_curr_prev(phi_ini, curr_ini, prev_ini, reference * np.pi / 180.0)
targ_last, rel_last, ref_last, errors_last = get_error_curr_prev(phi_last, curr_last, prev_last, reference * np.pi / 180.0)
print(targ_ini.shape, rel_ini.shape, ref_ini.shape, errors_ini.shape)
```

``` ipython
print(ref_ini.shape, rel_ini.shape, errors_ini.shape)
```

``` ipython
end_point_ini = get_end_point(model, errors_ini)
end_point_last = get_end_point(model, errors_last)
print(end_point_ini.shape, end_point_last.shape)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

ax[0].hist(reference[:, 0])
ax[1].hist(end_point_ini[:, 0], bins='auto')
ax[2].hist(end_point_last[:, 0], bins='auto')
plt.show()
```

``` ipython
time_points = np.linspace(0, model.DURATION, errors_ini.shape[-1])
idx = np.random.randint(errors_ini.shape[1], size=100)

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
ax[0].plot(time_points, errors_ini[0][idx].T, alpha=.4)
add_vlines(model, ax[0])

ax[0].set_xlabel('t')
ax[0].set_ylabel('error first half(°)')

ax[1].plot(time_points, errors_last[0][idx].T, alpha=.4)
add_vlines(model, ax[1])

ax[1].set_xlabel('t')
ax[1].set_ylabel('error last half (°)')
plt.show()
```

``` ipython
```

Serial Bias Curves
==================

``` ipython
print(targ_ini.shape, rel_ini.shape, ref_ini.shape, end_point_ini.shape)
```

``` ipython
n_bins = 16
data_ini = pd.DataFrame({'target_loc': targ_ini[:, -1], 'rel_loc': rel_ini[:, -1], 'ref_loc': ref_ini[:, -1], 'errors': end_point_ini[:, 0]})
data_last = pd.DataFrame({'target_loc': targ_last[:, -1], 'rel_loc': rel_last[:, -1], 'ref_loc': ref_last[:, -1], 'errors': end_point_last[:, 0]})
```

``` ipython
def get_correct_error(n_bins, df, error_type='rel_loc', thresh=25):
    import numpy as np
    import pandas as pd

    # 1. Threshold errors
    if thresh is not None:
        data = df[(df['errors'] >= -thresh) & (df['errors'] <= thresh)].copy()
    else:
        data = df.copy()

    # 2. Bin target locations
    bin_edges = np.linspace(0, 360, n_bins + 1)
    data['bin_target'] = pd.cut(data['target_loc'], bins=bin_edges, include_lowest=True)
    mean_errors_per_bin = data.groupby('bin_target')['errors'].mean()

    # 3. Remove mean error per target location (for rel_loc)
    if error_type == 'rel_loc':
        data['adjusted_errors'] = data['errors'] - data['bin_target'].map(mean_errors_per_bin).astype(float)
    else:
        data['adjusted_errors'] = data['errors']

    # 4. Bin by error_type for both full versions
    data['bin_error'] = pd.cut(data[error_type], bins=n_bins)
    bin_error = data.groupby('bin_error')['adjusted_errors'].agg(['mean', 'sem']).reset_index()
    edges = bin_error['bin_error'].cat.categories
    centers = (edges.left + edges.right) / 2

    # 5. Flipped error absolute analysis
    if error_type == 'rel_loc':
        # Bin abs(rel_loc) from 0 to 180
        data['error_abs'] = np.abs(data[error_type])
        data['bin_error_abs'] = pd.cut(data['error_abs'], bins=n_bins, include_lowest=True)
        # Flip so all directions use same sign
        data['adjusted_errors_abs'] = data['adjusted_errors'] * np.sign(data[error_type])
    else:
        # Bin abs(ref_loc) from 0 to 90
        data['error_abs'] = np.abs(data[error_type])
        data = data[data['error_abs'] <= 90.0]  # Only 0-90
        data['bin_error_abs'] = pd.cut(data['error_abs'], bins=n_bins, include_lowest=True)
        # Flip so all directions use same sign for ref_loc
        data['adjusted_errors_abs'] = data['adjusted_errors'] * np.sign(data[error_type])

    bin_error_abs = data.groupby('bin_error_abs')['adjusted_errors_abs'].agg(['mean', 'sem']).reset_index()
    edges_abs = bin_error_abs['bin_error_abs'].cat.categories
    centers_abs = (edges_abs.left + edges_abs.right) / 2

    return centers, bin_error, centers_abs, bin_error_abs
```

``` ipython
centers_ini, bin_rel_ini, centers_abs_ini, bin_rel_abs_ini = get_correct_error(n_bins, data_ini)
centers_last, bin_rel_last, centers_abs_last, bin_rel_abs_last = get_correct_error(n_bins, data_last)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(centers_ini, bin_rel_ini['mean'], 'r', label='First half')
ax[0].fill_between(centers_ini, bin_rel_ini['mean'] - bin_rel_ini['sem'], bin_rel_ini['mean'] + bin_rel_ini['sem'], color='r', alpha=0.2)

ax[0].plot(centers_last, bin_rel_last['mean'], 'b', label='Last half')
ax[0].fill_between(centers_last, bin_rel_last['mean'] - bin_rel_last['sem'], bin_rel_last['mean'] + bin_rel_last['sem'], color='b', alpha=0.2)

ax[0].axhline(0, color='k', linestyle=":")
ax[0].set_xlabel('Rel. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].set_xticks(np.linspace(-180, 180, 5))

ax[1].plot(centers_abs_ini, bin_rel_abs_ini['mean'], 'r', label='First half')
ax[1].fill_between(centers_abs_ini, bin_rel_abs_ini['mean'] - bin_rel_abs_ini['sem'], bin_rel_abs_ini['mean'] + bin_rel_abs_ini['sem'], color='r', alpha=0.2)

ax[1].plot(centers_abs_last, bin_rel_abs_last['mean'], 'b', label='Last half')
ax[1].fill_between(centers_abs_last, bin_rel_abs_last['mean'] - bin_rel_abs_last['sem'], bin_rel_abs_last['mean'] + bin_rel_abs_last['sem'], color='b', alpha=0.2)

ax[1].axhline(0, color='k', linestyle=":")
ax[1].set_xlabel('Rel. Loc. (°)')
ax[1].set_ylabel('Flip. Error (°)')
ax[1].legend(fontsize=12)
ax[1].set_xticks(np.linspace(0, 180, 3))

plt.tight_layout()
plt.show()
```

``` ipython
centers_ref_ini, bin_ref_ini, centers_ref_abs_ini, bin_ref_abs_ini = get_correct_error(n_bins, data_ini, error_type='ref_loc')
centers_ref_last, bin_ref_last, centers_ref_abs_last, bin_ref_abs_last = get_correct_error(n_bins, data_last, error_type='ref_loc')
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(centers_ref_ini, bin_ref_ini['mean'], 'r', label='First half')
ax[0].fill_between(centers_ref_ini, bin_ref_ini['mean'] - bin_ref_ini['sem'], bin_ref_ini['mean'] + bin_ref_ini['sem'], color='r', alpha=0.2)

ax[0].plot(centers_ref_last, bin_ref_last['mean'], 'b', label='Last half')
ax[0].fill_between(centers_ref_last, bin_ref_last['mean'] - bin_ref_last['sem'], bin_ref_last['mean'] + bin_ref_last['sem'], color='b', alpha=0.2)

ax[0].axhline(0, color='k', linestyle=":")
ax[0].set_xlabel('Ref. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].set_xticks(np.linspace(-180, 180, 5))

ax[1].plot(centers_ref_abs_ini, bin_ref_abs_ini['mean'], 'r', label='First half')
ax[1].fill_between(centers_ref_abs_ini, bin_ref_abs_ini['mean'] - bin_ref_abs_ini['sem'], bin_ref_abs_ini['mean'] + bin_ref_abs_ini['sem'], color='r', alpha=0.2)

ax[1].plot(centers_ref_abs_last, bin_ref_abs_last['mean'], 'b', label='Last half')
ax[1].fill_between(centers_ref_abs_last, bin_ref_abs_last['mean'] - bin_ref_abs_last['sem'], bin_ref_abs_last['mean'] + bin_ref_abs_last['sem'], color='b', alpha=0.2)

ax[1].axhline(0, color='k', linestyle=":")
ax[1].set_xlabel('Ref. Loc. (°)')
ax[1].set_ylabel('Flip. Error (°)')
ax[1].legend(fontsize=12)
ax[1].set_xticks(np.linspace(0, 90, 3))

plt.tight_layout()
plt.show()
```

``` ipython
```

Bias Evolution along a session
==============================

``` ipython
print(rates_list.shape)
```

``` ipython
_, _, phi_list = decode_bump_torch(rates_list, axis=-1)
print(phi_list.shape)
```

``` ipython
cmap = plt.get_cmap('Blues')
colors = [cmap( (i+1) / phi_list.shape[0] ) for i in range(phi_list.shape[0])]

n_bins = 16

serial_list = []
ref_list = []

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

for i in range(phi_list.shape[0]): # trial by trial
    targ_trial, rel_trial, ref_trial, errors_trial = get_error_curr_prev(phi_list[i, np.newaxis], curr_list[i, np.newaxis], prev_list[i, np.newaxis], reference * np.pi / 180.0)

    end_point_trial = get_end_point(model, errors_trial)

    # print(targ_trial.shape, rel_trial.shape, ref_trial.shape, errors_trial.shape, end_point_trial.shape)

    data = pd.DataFrame({'target_loc': targ_trial[:, -1], 'rel_loc': rel_trial[:, -1], 'ref_loc': ref_trial[:, -1], 'errors': end_point_trial[:, 0]})

    centers, bin_rel, centers_abs, bin_rel_abs = get_correct_error(n_bins, data)
    centers_ref, bin_ref, centers_abs, bin_ref_abs = get_correct_error(n_bins, data, error_type='ref_loc')

    ax[0].plot(centers, bin_rel['mean'], color=colors[i], alpha=1)
    ax[0].axhline(0, ls='--', color='k')
    ax[0].set_xlabel('Rel. Loc. (°)')
    ax[0].set_ylabel('Error (°)')
    ax[0].set_xticks(np.linspace(-180, 180, 5))

    ax[1].plot(centers_ref, bin_ref['mean'], color=colors[i], alpha=1)
    ax[1].axhline(0, ls='--', color='k')
    ax[1].set_xlabel('Ref. Loc. (°)')
    ax[1].set_ylabel('Error (°)')
    ax[1].set_xticks(np.linspace(-180, 180, 5))

    # idx_max = np.argmax(np.abs(bin_rel['mean'][centers>0]))
    # dum = bin_rel['mean'][centers>0]
    # serial_max = dum.iloc[idx_max]
    # dum = bin_rel['sem'][centers>0]
    # serial_std = dum.iloc[idx_max]

    idx_max = np.argmax(abs(bin_rel_abs['mean']))
    serial_max = bin_rel_abs['mean'][idx_max]
    serial_std = bin_rel_abs['sem'][idx_max]
    serial_list.append([serial_max, serial_std])

    idx_max = np.argmax(abs(bin_ref_abs['mean']))
    ref_max = bin_ref_abs['mean'][idx_max]
    ref_std = bin_ref_abs['sem'][idx_max]

    ref_list.append([ref_max, ref_std])

serial_list = np.array(serial_list).T
ref_list = np.array(ref_list).T
print(serial_list.shape)
plt.show()
```

``` ipython
delay_duration = 100
xdelay = np.linspace(0, delay_duration, serial_list.shape[1])

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

from scipy.ndimage import uniform_filter1d

s0 = 15
ax[0].plot(xdelay, uniform_filter1d(serial_list[0], s0), '-')
ax[0].fill_between(xdelay, uniform_filter1d(serial_list[0] - serial_list[1], s0), uniform_filter1d(serial_list[0] + serial_list[1], s0), color='b', alpha=0.2)
ax[0].axhline(0, ls='--', color='k')

ax[0].set_xlabel('Trial #')
ax[0].set_ylabel('Serial Bias (°)')

ax[1].plot(xdelay, uniform_filter1d(ref_list[0], s0), '-')
ax[1].fill_between(xdelay, uniform_filter1d(ref_list[0] - ref_list[1], s0), uniform_filter1d(ref_list[0] + ref_list[1], s0), color='b', alpha=0.2)
ax[1].axhline(0, ls='--', color='k')

ax[1].set_xlabel('Trial #')
ax[1].set_ylabel('Ref. Bias (°)')

plt.show()
```

``` ipython
```